In [ ]:
import os
os.chdir("..")

In [ ]:
import world_bank_data as wb
import pandas as pd
import polars as pl
import numpy as np
import pandas as pd
from src.data.data_pull import DataClean

dc = DataClean()

# TODO
- colapse by 5 average
- keep the balance data
- 

In [ ]:
dc.pull_wb()

In [ ]:
data = pl.read_csv("data/external/countries.csv")
data.count()

In [ ]:
df = dc.pull_wb()
df  = df.with_columns(group=pl.col("year")//5 * 5)
countries = df.select(pl.col("group")).to_series().unique().to_list()
df = df.group_by(["country", "group"]).agg(
    pl.col("fertility_rate").mean(), 
    pl.col("gdp_per_capita").mean(), 
    pl.col("government_expenditure").mean(), 
    pl.col("trade").mean(), 
    pl.col("life_expectancy").mean(), 
    pl.col("gdp_per_capita_growth").mean(), 
    pl.col("inflation").mean(), 
    pl.col("population_growth").mean(), 
    pl.col("school_primary").mean(), 
    pl.col("school_secondary").mean(), 
    pl.col("school_tertiary").mean(), 
    pl.col("gross_capital_formation").mean(), 
    pl.col("rule_of_law").mean(), 
    pl.col("control_of_corruption").mean(), 
    pl.col("regulatory_quality").mean(), 
    pl.col("political_stability").mean(), 
    pl.col("voice").mean(), 
    pl.col("government_effect").mean(), 
    )
df = df.join(data, on="country", how="inner", validate="m:1")
df = df.filter(pl.col("government_effect").is_not_null())
for year in countries:
    tmp = df.filter(pl.col("group") == year).select(pl.col("country")).unique().count().item()
    print(f"{year} count {tmp}")

In [ ]:
df = dc.pull_wb()
df = df.join(data, on="country", how="inner", validate="m:1")
df  = df.with_columns(group=pl.col("year")//5 * 5)
countries = df.select(pl.col("year")).to_series().unique().to_list()
data = pl.read_csv("data/external/countries.csv")
for cols in  df.columns:
    df = df.filter(pl.col(f"{cols}").is_not_null())

for year in countries:
    tmp = df.filter(pl.col("group") == year).select(pl.col("country")).unique().count().item()
    print(f"{year} count {tmp}")
# df  = df.sort(by=["country", "year"])
df.write_csv("data.csv")

In [ ]:
df

In [ ]:
df.write_csv("test.csv")

In [ ]:
df

In [ ]:
df.filter(pl.col("group") == 1965)

In [ ]:
df.select(pl.col("country")).unique().count().item()

In [ ]:
df = pl.DataFrame(wb.get_countries())
df

In [ ]:
df = dc.wb_data(params=["GFDD.OI.01"],year=1980)
df

In [ ]:
columns = [
    'gdp_growth', 'spending', 'gini',
    'expenses'
]
data = df.to_pandas()

# Step 1: Sort the DataFrame
data = data[data["country"] != "World"]
data = data[data["country"] == "United States"]
data = data.sort_values(by=['year', 'country']).reset_index(drop=True)

# Step 2: Interpolate each column by zip group
# data[columns] = data.groupby('country')[columns].transform(
#     lambda group: group.interpolate(method='linear', limit_direction='both')
# )
data = data.dropna(subset=["gdp_growth"]).reset_index(drop=True)
data = data.dropna(subset=["expenses"]).reset_index(drop=True)
data

In [ ]:
def plot_knots(knots, ax):
    for knot in knots:
        ax.axvline(knot, color="0.1", alpha=0.4)
    return ax

In [ ]:
num_knots = 5
knots = np.quantile(data["expenses"], np.linspace(0, 1, num_knots))
iknots = knots[1:-1]
priors = {
    "Intercept": bmb.Prior("Normal", mu=100, sigma=10),
    "common": bmb.Prior("Normal", mu=0, sigma=10), 
    "sigma": bmb.Prior("Exponential", lam=1)
}
model = bmb.Model("gdp_growth ~ year + bs(expenses, knots=iknots, intercept=True)", data, dropna=True, priors=priors)

In [ ]:
def plot_spline_basis(basis, expenses, figsize=(10, 6)):
    df = (
        pd.DataFrame(basis)
        .assign(expenses=expenses)
        .melt("expenses", var_name="basis_idx", value_name="gdp_growth")
    )

    _, ax = plt.subplots(figsize=figsize)

    for idx in df.basis_idx.unique():
        d = df[df.basis_idx == idx]
        ax.plot(d["expenses"], d["gdp_growth"])
    
    return ax

In [ ]:
B = model.components["mu"].design.common["bs(expenses, knots=iknots, intercept=True)"]
ax = plot_spline_basis(B, data["expenses"].values)
plot_knots(knots, ax)

In [ ]:
# The seed is to make results reproducible
idata = model.fit(idata_kwargs={"log_likelihood": True})

In [ ]:
az.plot_trace(idata);

In [ ]:
posterior_stacked = az.extract(idata)
wp = posterior_stacked["bs(expenses, knots=iknots, intercept=True)"].mean("sample").values

ax = plot_spline_basis(B * wp.T, data["expenses"].values)
ax.plot(data.expenses.values, np.dot(B, wp.T), color="black", lw=3)
plot_knots(knots, ax);